# IPython interface for running GPU DSM on cluster

In [77]:
from dsm_tools import ssh_init, job_start, jobs_done, job_transfer_clean, uuid, os, server, username, password
from fdt_fit import fdt_fit

** Choose input parameters for GPU DSM **

In [33]:
class Calculation:
    #Input parameters
    beta = 1
    nChains = 5000
    velocityGradient = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    CDtoggle = 0
    PDtoggle = 0
    calcMode = 1
    timeStep = 1
    simTime = 10000
    token = None #folder name
    ssh = None #storing Paramiko SSH object
    jobIds = None #array of job IDs on cluster
    jobStatus = 0
    #Calculation options (number of GPU)

    #Function to generate input file
    def generate_token(self):
        self.token = str(uuid.uuid4())
        return

    #Function to start calculation on cluster
    def calc(self):
        self.ssh=ssh_init()
        self.jobIds = job_start(self.ssh, self.token)
        self.jobStatus = 1
        return

    #Function to check the status of calculation
    def check_self(self):
        if not self.ssh:
            self.ssh=ssh_init()
        if jobs_done(self.ssh, self.jobIds):
            print('Calculation is done')
            job_transfer_clean(self.ssh, self.token)
        else:
            print('Still running, check back later')

    #Function to run f_d(t)

    #Initialize object of class
    def __init__(self):
        self.generate_token()
        if not os.path.exists(self.token):
            os.makedirs(self.token)
        self.generate_input()


class CalculationStar(Calculation):
    #Type of chain
    architecture = 'star'
    nArms = 3
    nkArms = [10, 10, 10]

    def generate_input(self):
        file = open(self.token + "/input.dat","w")

        file.write(str(self.beta)+'\n')
        file.write(self.architecture+'\n')
        file.write(str(self.nArms)+'\n')
        file.write(" ".join(map(str,self.nkArms)) + '\n')
        file.write(str(self.nChains)+'\n')
        file.write(" ".join(map(str,self.velocityGradient)) + '\n')
        file.write(str(self.CDtoggle)+'\n')
        file.write(str(self.PDtoggle)+'\n')
        file.write(str(self.calcMode)+'\n')
        file.write(str(self.timeStep)+'\n')
        file.write(str(self.simTime))

        file.close()

class CalculationLinear(Calculation):
    #Type of chain
    architecture = 'linear'
    nk = 10

    def generate_input(self):
        file = open(self.token + "/input.dat","w")

        file.write(str(self.beta)+'\n')
        file.write(self.architecture+'\n')
        file.write(str(self.nk)+'\n')
        file.write(str(self.nChains)+'\n')
        file.write(" ".join(map(str,self.velocityGradient)) + '\n')
        file.write(str(self.CDtoggle)+'\n')
        file.write(str(self.PDtoggle)+'\n')
        file.write(str(self.calcMode)+'\n')
        file.write(str(self.timeStep)+'\n')
        file.write(str(self.simTime))

        file.close()

In [34]:
x = CalculationLinear()

In [35]:
%%capture
x.calc()

In [37]:
x.check_self()

In [43]:
x.token

In [44]:
cd cf074bc9-9550-49a4-bb66-00ef825dff66

In [78]:
fdt_fit()

In [30]:
beta = 1
architecture = 'linear'
#nArms = 3
nkArms = [45]
nChains = 5000
velocityGradient = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
CDtoggle = 0
PDtoggle = 0
calcMode = 1
timeStep = 1
simTime = 100000